In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
#homogeneous process
"""""
This function simulates homogeneous Poisson process, counting event arrival.

Inputs:

lamba_type (float): A non negative scalar value. It represents the average rate of intensity of the events being simulated
T (float): This is a positive scalar value. It is the obeserved time horizon or duration of the simulation.
N (int): Positive integer representing the number of independent processes or entitities to simulate

Output:

3D Array containing the simulated event arrival times and entity IDs for each entity. 
"""""


def sim_hpp(lambda_type, T, N):
    if lambda_type <= 0 or not np.isscalar(lambda_type):
        raise ValueError("simHPP: Lambda must be a positive scalar.")
    if T <= 0 or not np.isscalar(T):
        raise ValueError("simHPP: T must be a positive scalar.")
    if N <= 0 or not np.isscalar(N):
        raise ValueError("simHPP: N must be a positive scalar.")

    EN = np.random.poisson(lambda_type * T, N)
    max_EN = 2 * np.max(EN) + 2
    y = np.full((max_EN, N, 2), T)
    y[:, :, 1] = np.tile(EN, (max_EN, 1))
    
    for i in range(N):
        if EN[i] > 0:
            ttmp = np.sort(T * np.random.uniform(size=EN[i]))
            y[:2 * EN[i] + 1, i, 0] = np.concatenate(([0], np.repeat(ttmp, 2)))
            y[:2 * EN[i] + 1, i, 1] = np.concatenate(([0], np.floor(np.arange(1, 2 * EN[i]) / 2), [EN[i]]))

    return y

In [21]:
def sim_nhpp(lambda_type, parlambda, T, N):
    a = parlambda[0]
    b = parlambda[1]
    if lambda_type == 0:
        d = parlambda[2]
        JM = sim_hpp(a + b, T, N)
    elif lambda_type == 1:
        JM = sim_hpp(a + b * T, T, N)
    elif lambda_type == 2:
        d = parlambda[2]
        JM = sim_hpp(a + b * T, T, N)
    
    rjm = JM.shape[0]
    yy = np.zeros((rjm, N, 2))
    yy[:, :, 0] = T
    
    max_EN = 0
    for i in range(N):
        pom = JM[JM[:, i, 0] < T, i, 0]
        pom = pom[1::2]
        R = np.random.uniform(size=len(pom))
        
        if lambda_type == 0:
            lambdat = (a + b * np.sin(2 * np.pi * (pom + d))) / (a + b)
        elif lambda_type == 1:
            lambdat = (a + b * pom) / (a + b * T)
        elif lambda_type == 2:
            lambdat = (a + b * np.sin(2 * np.pi * (pom + d)) ** 2) / (a + b)
        
        pom = pom[R < lambdat]
        EN = len(pom)
        max_EN = max(max_EN, EN)
        yy[:2 * EN + 1, i, 0] = np.concatenate(([0], np.repeat(pom, 2)))
        yy[2:2 * EN + 1, i, 1] = np.floor(np.arange(1, 2 * EN) / 2)
        yy[2 * EN + 1:, i, 1] = EN
    
    return yy[:2 * max_EN + 2]


In [22]:
N = 4
T = 2
lambda_type = 1
parlambda = [1, 2]


In [23]:
samples = sim_nhpp(lambda_type, parlambda, T, N)